## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-21-14-01-00 +0000,world,Opinion | Toppling Tehran Isn’t Like Sacking S...,https://www.wsj.com/opinion/toppling-tehran-is...
1,2025-07-21-11-53-00 +0000,world,China Confirms Exit Ban on Wells Fargo Banker,https://www.wsj.com/world/china/china-confirms...
2,2025-07-21-03-00-00 +0000,world,"The Fiercest Border Fight: High Kicks, Mustach...",https://www.wsj.com/world/the-fiercest-border-...
3,2025-07-21-03-00-00 +0000,world,Germany Shut Down Thousands of Air-Raid Shelte...,https://www.wsj.com/world/europe/germany-new-a...
4,2025-07-21-02-01-00 +0000,world,Opinion | The Populist Wave Hits Japan,https://www.wsj.com/opinion/the-populist-wave-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2293/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,fight,2
30,new,2
36,japan,2
47,trade,2
2,isn,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-07-21-03-00-00 +0000,world,"The Fiercest Border Fight: High Kicks, Mustach...",https://www.wsj.com/world/the-fiercest-border-...,11
3,2025-07-21-03-00-00 +0000,world,Germany Shut Down Thousands of Air-Raid Shelte...,https://www.wsj.com/world/europe/germany-new-a...,11
5,2025-07-21-01-00-00 +0000,world,Why Israel’s Chaotic New Food Program in Gaza ...,https://www.wsj.com/world/middle-east/us-israe...,10
7,2025-07-21-00-16-00 +0000,world,Japan Election Throws a Wrench in Trade Talks,https://www.wsj.com/world/asia/japan-election-...,8
1,2025-07-21-11-53-00 +0000,world,China Confirms Exit Ban on Wells Fargo Banker,https://www.wsj.com/world/china/china-confirms...,7


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,11,2025-07-21-03-00-00 +0000,world,"The Fiercest Border Fight: High Kicks, Mustach...",https://www.wsj.com/world/the-fiercest-border-...
3,11,2025-07-21-03-00-00 +0000,world,Germany Shut Down Thousands of Air-Raid Shelte...,https://www.wsj.com/world/europe/germany-new-a...
5,8,2025-07-21-01-00-00 +0000,world,Why Israel’s Chaotic New Food Program in Gaza ...,https://www.wsj.com/world/middle-east/us-israe...
7,8,2025-07-21-00-16-00 +0000,world,Japan Election Throws a Wrench in Trade Talks,https://www.wsj.com/world/asia/japan-election-...
1,7,2025-07-21-11-53-00 +0000,world,China Confirms Exit Ban on Wells Fargo Banker,https://www.wsj.com/world/china/china-confirms...
0,6,2025-07-21-14-01-00 +0000,world,Opinion | Toppling Tehran Isn’t Like Sacking S...,https://www.wsj.com/opinion/toppling-tehran-is...
8,6,2025-07-20-21-12-00 +0000,world,Opinion | Predatory Sparrow Hacks Iran’s Finan...,https://www.wsj.com/opinion/predatory-sparrow-...
4,3,2025-07-21-02-01-00 +0000,world,Opinion | The Populist Wave Hits Japan,https://www.wsj.com/opinion/the-populist-wave-...
6,2,2025-07-21-01-00-00 +0000,world,Europe Prepares for a U.S. Trade Fight,https://www.wsj.com/world/europe/europe-prepar...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
